## 1A) Feature detection and feature extraction

This notebook is an introduction to feature detection and extraction. Features are "interesting" parts of an image, and they are the basis of many computer vision algorithms like image matching, object detection, and 3D reconstruction. We will explore two main concepts:
- **Feature Detection**: Identifying points of interest (keypoints) in an image.
- **Feature Description**: Creating a vector representation (descriptor) of the local neighborhood around a keypoint.

We will look at classic algorithms like Harris Corner Detector, SIFT, and ORB.

In [ ]:
# JUST RUN THIS CELL

%load_ext autoreload
%autoreload 2

%matplotlib inline

import cv2 # opencv is a computer vision library
import numpy as np # numpy is a library for numerical computing
from matplotlib import pyplot as plt # matplotlib is a plotting library
import pathlib
import tqdm

import sys
import os
sys.path.append('../../')

import utils
from utils import show_image, show_images, non_maxima_suppression

cwd = os.getcwd()
feature_1_block_dir = pathlib.Path(os.getcwd()).parent
images_path = feature_1_block_dir / 'images'

In [ ]:
# JUST RUN THIS CELL

utils.CV2_MAX_IMAGE_HEIGHT = 1080/2  # set a maximum height for cv2 image visualization
utils.SHOW_IMAGE_BACKEND = utils.Backend.PLT  # set the default backend for visualization

Load the provided image. If you want to use a different image for this python exercise, add it to the `images` folder.

In [ ]:
# JUST RUN THIS CELL

IMAGE_NAME = 'west-village.jpg'

RESIZE_FACTOR = 0.33

image_path = images_path / IMAGE_NAME

input_image = cv2.imread(str(image_path))

if RESIZE_FACTOR != 1.0:
	input_image = cv2.resize(input_image, (0,0), fx=RESIZE_FACTOR, fy=RESIZE_FACTOR)

input_gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

show_images([input_image, input_gray_image], ["Input Image", "Input Gray Image"])

### Harris corner detector

The Harris corner detector is a classic algorithm for finding corners in an image. Corners are regions in the image with large intensity variation in all directions. The algorithm works by analyzing the local changes in intensity around each pixel.

The main idea is to compute a "cornerness" score for each pixel. A high score indicates a corner. This is done by looking at a small window around the pixel and seeing how much the image changes when the window is shifted in different directions.

The code below first implements the Harris corner detector from scratch, and then uses the OpenCV implementation. You can see how the gradients `Ix` and `Iy` are computed, and then combined to form the structure tensor `M`, from which the Harris response is calculated.

Your task in implementing the Harris corner detection:
1) Fix the `TODO_compute_y_gradient` function to compute the x gradient of an image.
2) Fix the `TODO_compute_harris_corner_response` function with the actual formula to compute the harris response (cornerness score).
3) Play with the capital case parameters to refine the corner detections.

In [ ]:
#### PLAY WITH THESE PARAMETERS TO REFINE THE HARRIS CORNER DETECTION ####
MANUAL_HARRIS_K = 0.01
MANUAL_HARRIS_BLOCK_SIZE = 5

VIZ_MANUAL_HARRIS_RELATIVE_RESPONSE_THRESHOLD = 0.01
##########################################

# TODO: fix the functions below
def TODO_compute_y_gradient(grayscale_image: np.ndarray) -> np.ndarray:
	filter_y_grad = np.array([[-1, 0, 1]])
	Iy = cv2.filter2D(grayscale_image, -1, filter_y_grad)
	return Iy

# TODO: implement the functions below
def TODO_compute_harris_corner_response(Sxx: np.ndarray, Syy: np.ndarray, Sxy: np.ndarray, k=MANUAL_HARRIS_K) -> np.ndarray:
	# # R = det(M) - k * trace(M)^2
	# # where M = [[Sxx, Sxy], [Sxy, Syy]]
	det_M = (Sxx * Syy) - (Sxy ** 2)
	trace_M = Sxx + Syy
	TODO_harris_response = det_M - k * (trace_M ** 2)
	# TODO_harris_response = Sxx**2 * Syy**2 - (Sxy ** k)
	return TODO_harris_response


def my_harris_corner_detector(image, k=MANUAL_HARRIS_K, blockSize=MANUAL_HARRIS_BLOCK_SIZE, verbose=False):

	if image.ndim == 3:
		grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	else:
		grayscale_image = image.copy()

	# Step 1: Compute image gradients	
	filter_x = np.array([[-1, 0, 1]])
	Ix = cv2.filter2D(grayscale_image.astype(np.float32), -1, filter_x)
	Iy = TODO_compute_y_gradient(grayscale_image.astype(np.float32))

	if verbose:
		Ix_scaled = cv2.applyColorMap(cv2.normalize(Ix, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8), cv2.COLORMAP_TWILIGHT_SHIFTED)
		Iy_scaled = cv2.applyColorMap(cv2.normalize(Iy, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8), cv2.COLORMAP_TWILIGHT_SHIFTED)
		show_images([Ix_scaled, Iy_scaled], ["Ix (Gradient X)", "Iy (Gradient Y)"])

	# Step 2: Compute products of derivatives at every pixel
	Ixx = Ix ** 2 # square of horizontal gradient
	Iyy = Iy ** 2 # square of vertical gradient
	Ixy = Ix * Iy # product of gradients

	if verbose:
		Ixx_scaled = cv2.applyColorMap(cv2.normalize(Ixx, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8), cv2.COLORMAP_TWILIGHT_SHIFTED)
		Iyy_scaled = cv2.applyColorMap(cv2.normalize(Iyy, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8), cv2.COLORMAP_TWILIGHT_SHIFTED)
		Ixy_scaled = cv2.applyColorMap(cv2.normalize(Ixy, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8), cv2.COLORMAP_TWILIGHT_SHIFTED)
		show_images([Ixx_scaled, Iyy_scaled, Ixy_scaled], ["Ixx", "Iyy", "Ixy"], cmap='Reds')

	# Step 3: Compute sums of products of derivatives
	# Trick: use a Gaussian filter to compute the sum of products within a window
	# weighting towards the center of the window
	kernel = cv2.getGaussianKernel(blockSize, sigma=-1)
	kernel = kernel @ kernel.T

	Sxx = cv2.filter2D(Ixx, -1, kernel) # sum of Ixx within window
	Syy = cv2.filter2D(Iyy, -1, kernel) # sum of Iyy within window
	Sxy = cv2.filter2D(Ixy, -1, kernel) # sum of Ixy within window

	if verbose:
		Sxx_scaled = cv2.applyColorMap(cv2.normalize(Sxx, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8), cv2.COLORMAP_TWILIGHT_SHIFTED)
		Syy_scaled = cv2.applyColorMap(cv2.normalize(Syy, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8), cv2.COLORMAP_TWILIGHT_SHIFTED)
		Sxy_scaled = cv2.applyColorMap(cv2.normalize(Sxy, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8), cv2.COLORMAP_TWILIGHT_SHIFTED)
		show_images([Sxx_scaled, Syy_scaled, Sxy_scaled], ["Sxx", "Syy", "Sxy"], cmap='Reds')

	# Step 4: Compute Harris corner response
	harris_response = TODO_compute_harris_corner_response(Sxx, Syy, Sxy, k=MANUAL_HARRIS_K)

	# Visualize the result
	if verbose:
		# harris_response_viz = cv2.normalize(harris_response, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
		harris_response_viz = harris_response.copy()
		harris_viz = image.copy()
		harris_viz = cv2.cvtColor(harris_viz, cv2.COLOR_GRAY2BGR)
		harris_response_viz[harris_response_viz < VIZ_MANUAL_HARRIS_RELATIVE_RESPONSE_THRESHOLD * harris_response_viz.max()] = 0
		harris_viz[harris_response_viz > 0] = [0, 255, 0]  # mark corners in green
		harris_viz = cv2.normalize(harris_viz, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

		harris_response_viz = cv2.normalize(harris_response, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
		harris_response_viz = cv2.applyColorMap(harris_response_viz, cv2.COLORMAP_TWILIGHT_SHIFTED)

		show_images([harris_response_viz, harris_viz], ["Manual Harris Corner Response\n(Sxx * Syy) - (Sxy²) - k * (Sxx + Syy)²", "Manual Harris Corner Visualization"])

		print(f"Manual Harris corner detection completed.")

	return harris_response

harris_response = my_harris_corner_detector(input_gray_image, verbose=True)

Here you can decide to run your Harris detector (`USE_MY_HARRIS_DETECTOR=True`) or the cv2 Harris detector. Play with the parameters to refine the corner detection.

In [ ]:
USE_MY_HARRIS_DETECTOR = True

HARRIS_RESIZE_FACTOR = 1.0  # < 1.0 will downscale the image

#### PLAY WITH THESE PARAMETERS TO REFINE THE HARRIS CORNER DETECTION ####
CV2_HARRIS_KERNEL_SIZE = 3

HARRIS_BLOCK_SIZE = 3
HARRIS_K = 0.05
HARRIS_CORNER_RESPONSE_THRESHOLD = 0.01
HARRIS_MAX_FEATURES = 40000
########################################

HARRIS_RICH_VISUALIZATION = True
HARRIS_VIZ_KEYPOINT_SIZE = 5
HARRIS_KEYPOINT_COLOR = (0, 255, 0) # (B,G,R) color tuple or None for random color per keypoint

harris_input_image = cv2.resize(input_image, (0,0), fx=HARRIS_RESIZE_FACTOR, fy=HARRIS_RESIZE_FACTOR)
harris_input_gray_image = cv2.resize(input_gray_image, (0,0), fx=HARRIS_RESIZE_FACTOR, fy=HARRIS_RESIZE_FACTOR)

# run the harris corner detector
if USE_MY_HARRIS_DETECTOR:
    dst = my_harris_corner_detector(harris_input_gray_image, k=HARRIS_K, blockSize=HARRIS_BLOCK_SIZE, verbose=False)
else:
    dst = cv2.cornerHarris(harris_input_gray_image, HARRIS_BLOCK_SIZE, CV2_HARRIS_KERNEL_SIZE, HARRIS_K)
    
#result is dilated for marking the corners, this is visualisation related and just makes them bigger
dst[dst < HARRIS_CORNER_RESPONSE_THRESHOLD * dst.max()] = 0

keypoints = np.argwhere(dst > HARRIS_CORNER_RESPONSE_THRESHOLD * dst.max())
harris_cv2_keypoints = [cv2.KeyPoint(x=float(kp[1]), y=float(kp[0]),
                 size=HARRIS_VIZ_KEYPOINT_SIZE, response=dst[kp[0], kp[1]]) for kp in keypoints]

print(f"Detected {len(harris_cv2_keypoints)} Harris keypoints")

# perform non-maxima suppression to keep only the strongest keypoints in a local neighbourhood
harris_cv2_keypoints = non_maxima_suppression(harris_cv2_keypoints, input_image.shape, windowsize=7)

# sort keypoints by response and keep the best N
harris_cv2_keypoints = sorted(harris_cv2_keypoints, key=lambda kp: kp.response, reverse=True)[:HARRIS_MAX_FEATURES]

print(f"Keeping the top {len(harris_cv2_keypoints)} Harris keypoints")

harris_viz_flag = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS if HARRIS_RICH_VISUALIZATION else 0
harris_viz_out = cv2.drawKeypoints(harris_input_image, harris_cv2_keypoints, None, color=HARRIS_KEYPOINT_COLOR, flags=harris_viz_flag)
# dst = cv2.dilate(dst, kernel=np.ones((3, 3), np.uint8), iterations=8)
# #we then plot these on the input image for visualisation purposes, using bright red
# harris_viz_out = input_image.copy()
# harris_viz_out[dst > CORNER_RESPONSE_THRESHOLD * dst.max()] = [255,0,2055]

show_image(harris_viz_out, "Harris Corners")

The centre of each circle is the keypoint location, the line from the centre to the circle is the orientation of the keypoint, and the size of the circle is the scale at which this feature was detected.

In [ ]:
# Choose a keypoint to visualize
HARRIS_KEYPOINT_IDX = 10

if HARRIS_KEYPOINT_IDX >= len(harris_cv2_keypoints):
	raise ValueError(f"HARRIS_KEYPOINT_IDX {HARRIS_KEYPOINT_IDX} is out of bounds"
				  f" for number of keypoints {len(harris_cv2_keypoints)}")

harris_keypoint = harris_cv2_keypoints[HARRIS_KEYPOINT_IDX]
out_img_1_desc = input_image.copy()
cv2.drawKeypoints(out_img_1_desc, [harris_keypoint], out_img_1_desc, color=HARRIS_KEYPOINT_COLOR, flags=harris_viz_flag)

# now crop image around keypoint
x, y = harris_keypoint.pt
s = min(*input_image.shape[:2], 100)
min_x = max(0, int(x - s/2))
min_y = max(0, int(y - s/2))
max_x = min(input_image.shape[1], int(x + s/2))
max_y = min(input_image.shape[0], int(y + s/2))
crop_img = out_img_1_desc[min_y:max_y, min_x:max_x]
show_images([out_img_1_desc, crop_img], [f"HARRIS corner keypoint n. {HARRIS_KEYPOINT_IDX}", f"Cropped Image at keypoint n. {HARRIS_KEYPOINT_IDX}"])

### SIFT (Scale Invariant Feature Transform) detector

[SIFT](https://en.wikipedia.org/wiki/Scale-invariant_feature_transform) (Scale-Invariant Feature Transform) is robust and powerful feature detection and description algorithm. Unlike the Harris detector, SIFT is invariant to scale. This means it can detect the same feature in images even if it's bigger or smaller.

SIFT does not find corners only. SIFT finds areas that are significantly brighter or darker than their neighbors across different scales. This is done by applying the [Difference of Gaussians (DoG)](https://evidentscientific.com/en/microscope-resource/tutorials/digital-imaging/processing/diffgaussians) operator to an image pyramid to find local extrema, which allows it to find features at different scales. For each keypoint, it computes a descriptor that is a histogram of oriented gradients in the keypoint's neighborhood. This descriptor is what makes it robust to changes in illumination and viewpoint. However, this comes at a price of a slow eexcution speed compared to other feature detectors and descriptors.

The code below uses OpenCV's SIFT implementation to detect keypoints and compute their descriptors. The keypoints are visualized on the image. The size of the circle represents the scale at which the feature was detected, and the line indicates its orientation.

In [ ]:
### PLAY WITH THESE PARAMETERS TO REFINE THE SIFT FEATURE DETECTION AND DESCRIPTION ####
SIFT_N_FEATURES = None
SIFT_N_OCTAVE_LAYERS = 3
SIFT_CONTRAST_THRESHOLD = 0.04
SIFT_EDGE_THRESHOLD = 10
SIFT_SIGMA = 1.6
##########################################

SIFT_RICH_VISUALIZATION = True
SIFT_KEYPOINT_COLOR = None # (B,G,R) color tuple or None for random color per keypoint

sift_extractor = cv2.SIFT_create(
	nfeatures=SIFT_N_FEATURES,
	nOctaveLayers=SIFT_N_OCTAVE_LAYERS,
	contrastThreshold=SIFT_CONTRAST_THRESHOLD,
	edgeThreshold=SIFT_EDGE_THRESHOLD,
	sigma=SIFT_SIGMA
)

# extract keypoints
sift_keypoints = sift_extractor.detect(input_image, None)
print(f"Detected {len(sift_keypoints)} SIFT keypoints")

# compute the descriptors at the detected keypoints
sift_keypoints, sift_descriptors = sift_extractor.compute(input_image, sift_keypoints)

sift_img = input_image.copy()
sift_viz_flag = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS if SIFT_RICH_VISUALIZATION else 0
cv2.drawKeypoints(sift_img, sift_keypoints, sift_img, color=SIFT_KEYPOINT_COLOR, flags=sift_viz_flag)
show_image(sift_img, "SIFT Keypoints")

Now let's take a closer look at a single SIFT descriptor. Each SIFT descriptor is a 128-dimensional vector that encodes information about the gradients in the region around a keypoint. The descriptor captures the distribution of gradient orientations in a scale and rotation invariant way.

In the next cell, select a specific SIFT keypoint (sorted by size to better visualize larger-scale features). The surrounding image patch will be displayed, and its 128-dimensional descriptor vector will be printed.

In [ ]:
# Choose an index of a descriptor to visualize
SIFT_DESCRIPTOR_IDX_BY_SIZE = 3

if sift_descriptors is None:
    raise ValueError(f"No SIFT descriptors were computed.")

if SIFT_DESCRIPTOR_IDX_BY_SIZE >= len(sift_descriptors):
	raise ValueError(f"SIFT_DESCRIPTOR_IDX_BY_SIZE {SIFT_DESCRIPTOR_IDX_BY_SIZE} is out of bounds for number of descriptors {len(sift_descriptors)}")

size_sorted_sift_keypoints = sorted(sift_keypoints, key=lambda kp: kp.size, reverse=True)

if sift_descriptors is not None:
	print(f"SIFT descriptor {SIFT_DESCRIPTOR_IDX_BY_SIZE}: {sift_descriptors[SIFT_DESCRIPTOR_IDX_BY_SIZE]}")
	print(f"\nsize: {len(sift_descriptors[SIFT_DESCRIPTOR_IDX_BY_SIZE])}")

# now crop image around keypoint
x, y = size_sorted_sift_keypoints[SIFT_DESCRIPTOR_IDX_BY_SIZE].pt
s = max(size_sorted_sift_keypoints[SIFT_DESCRIPTOR_IDX_BY_SIZE].size, input_image.shape[0]/20, input_image.shape[1]/20)
s *= 2
min_x = max(0, int(x - s/2))
min_y = max(0, int(y - s/2))
max_x = min(input_image.shape[1], int(x + s/2))
max_y = min(input_image.shape[0], int(y + s/2))
out_img = input_image.copy()
out_img = cv2.drawKeypoints(out_img, [size_sorted_sift_keypoints[SIFT_DESCRIPTOR_IDX_BY_SIZE]],
	out_img, color=(0,0,255),
	flags=sift_viz_flag | cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS | cv2.DRAW_MATCHES_FLAGS_DRAW_OVER_OUTIMG)
out_img = cv2.circle(out_img, (int(x), int(y)), radius=3, color=(0,0,255), thickness=-1)
crop_img = out_img[min_y:max_y, min_x:max_x]
show_image(crop_img, f"Cropped Image at specific keypoint")

A descriptor is a vector of numbers that describes the area around a keypoint. The distance between two descriptors can be used as a measure of how similar the keypoints are. A small distance means the keypoints are likely to be a match. The L2 distance (or Euclidean distance) is a common way to measure this.

In the cell below, we calculate the L2 distance between the descriptors of two different SIFT keypoints. A large distance suggests that the local image patches around these two keypoints are very different.

In [ ]:
# Chose two descriptors to compare
FIRST_SIFT_DESCRIPTOR_IDX = 2
SECOND_SIFT_DESCRIPTOR_IDX = 3

if sift_descriptors is None:
    raise ValueError(f"No SIFT descriptors were computed.")

if FIRST_SIFT_DESCRIPTOR_IDX >= len(sift_descriptors) or SECOND_SIFT_DESCRIPTOR_IDX >= len(sift_descriptors):
    raise ValueError(f"One of the SIFT descriptor indices is out of bounds for number of descriptors {len(sift_descriptors)}")

difference = sift_descriptors[FIRST_SIFT_DESCRIPTOR_IDX] - sift_descriptors[SECOND_SIFT_DESCRIPTOR_IDX]
print(f"Difference between SIFT descriptors {FIRST_SIFT_DESCRIPTOR_IDX} and {SECOND_SIFT_DESCRIPTOR_IDX}: {difference}")

l2_distance = np.linalg.norm(difference)
print(f"\nL2 distance between SIFT descriptors {FIRST_SIFT_DESCRIPTOR_IDX} and {SECOND_SIFT_DESCRIPTOR_IDX}: {l2_distance:.2f}")

### ORB (Oriented FAST and Rotated BRIEF) detector

[ORB](https://en.wikipedia.org/wiki/Oriented_FAST_and_rotated_BRIEF) is a fast and efficient alternative to SIFT. It is a fusion of the [FAST](https://en.wikipedia.org/wiki/Features_from_accelerated_segment_test) keypoint detector and the [BRIEF](https://gilscvblog.com/2013/09/19/a-tutorial-on-binary-descriptors-part-2-the-brief-descriptor/) descriptor, with many modifications to enhance performance. ORB is rotation invariant and resistant to noise. It is also much faster than SIFT, making it suitable for real-time applications.

The code below uses OpenCV's ORB implementation to detect keypoints and compute their descriptors.

In [ ]:
### PLAY WITH THESE PARAMETERS TO REFINE THE ORB FEATURE DETECTION AND DESCRIPTION ####
ORB_N_FEATURES = 1000
ORB_SCALE_FACTOR = 1.2
ORB_N_LEVELS = 3
ORB_EDGE_THRESHOLD = 11
ORB_FIRST_LEVEL = 0
ORB_PATCH_SIZE = 31
ORB_FAST_THRESHOLD = 20
##########################################

ORB_RICH_VISUALIZATION = True
ORB_KEYPOINT_COLOR = None # (B,G,R) color tuple or None for random color per keypoint

orb_extractor = cv2.ORB_create(
    nfeatures=ORB_N_FEATURES,
    scaleFactor=ORB_SCALE_FACTOR,
    nlevels=ORB_N_LEVELS,
    edgeThreshold=ORB_EDGE_THRESHOLD,
    firstLevel=ORB_FIRST_LEVEL,
    patchSize=ORB_PATCH_SIZE,
    fastThreshold=ORB_FAST_THRESHOLD
)
# find the keypoints with ORB
orb_keypoints = orb_extractor.detect(input_image,None)

print(f"Detected {len(orb_keypoints)} ORB keypoints")

# compute the descriptors with ORB
orb_keypoints, orb_descriptors = orb_extractor.compute(input_image, orb_keypoints)
# draw keypoints
orb_img = input_image.copy()
flag_viz = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS if ORB_RICH_VISUALIZATION else 0
cv2.drawKeypoints(orb_img, orb_keypoints, orb_img, color=ORB_KEYPOINT_COLOR, flags=flag_viz)
show_image(orb_img, "ORB Keypoints")

Now let's take a closer look at a single ORB descriptor. Each ORB descriptor is a 128-dimensional vector that encodes information about the gradients in the region around a keypoint. The descriptor captures the distribution of gradient orientations in a scale and rotation invariant way.

In the next cell, select a specific ORB keypoint (sorted by size to better visualize larger-scale features). The surrounding image patch will be displayed, and its 128-dimensional descriptor vector will be printed.

In [ ]:
# Choose an index of a descriptor to visualize
ORB_DESCRIPTOR_IDX = 100

if orb_descriptors is None:
	raise ValueError(f"No ORB descriptors were computed.")

if ORB_DESCRIPTOR_IDX >= len(orb_descriptors):
	raise ValueError(f"ORB_DESCRIPTOR_IDX {ORB_DESCRIPTOR_IDX} is out of bounds for number of descriptors {len(orb_descriptors)}")

if orb_descriptors is not None:
  print(f"ORB descriptor {ORB_DESCRIPTOR_IDX}: {orb_descriptors[ORB_DESCRIPTOR_IDX]}")
  print(f"\nsize: {len(orb_descriptors[ORB_DESCRIPTOR_IDX])}")

# now crop image around keypoint
x, y = orb_keypoints[ORB_DESCRIPTOR_IDX].pt
s = max(orb_keypoints[ORB_DESCRIPTOR_IDX].size, input_image.shape[0]/20, input_image.shape[1]/20)
s *= 2
min_x = max(0, int(x - s/2))
min_y = max(0, int(y - s/2))
max_x = min(input_image.shape[1], int(x + s/2))
max_y = min(input_image.shape[0], int(y + s/2))
out_img = input_image.copy()
orb_viz_flag = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS if ORB_RICH_VISUALIZATION else 0
out_img = cv2.drawKeypoints(out_img, [orb_keypoints[ORB_DESCRIPTOR_IDX]],
	out_img, color=ORB_KEYPOINT_COLOR,
	flags=orb_viz_flag | cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS | cv2.DRAW_MATCHES_FLAGS_DRAW_OVER_OUTIMG)
crop_img = out_img[min_y:max_y, min_x:max_x]
show_image(crop_img, f"Cropped Image at keypoint n. {ORB_DESCRIPTOR_IDX}")

### Combining Detectors and Descriptors

It's possible to mix and match feature detectors and descriptors. For example, you can use a fast detector like ORB to find keypoints, and then use a more powerful (but slower) descriptor like SIFT to describe them. This can be a good compromise between speed and accuracy.

In the following cell, we detect keypoints using ORB and then compute both ORB and SIFT descriptors for these same keypoints. This allows us to have two different representations for the same set of interest points.

In [ ]:
# JUST RUN THIS CELL

orb_keypoints_2 = orb_extractor.detect(input_image,None)

print(f"Detected {len(orb_keypoints_2)} ORB keypoints")

_, orb_descriptors_2 = orb_extractor.compute(input_image, orb_keypoints_2)

print(f"Computed {len(orb_descriptors_2)} ORB descriptors at ORB keypoint locations")

_, sift_descriptors_2 = sift_extractor.compute(input_image, orb_keypoints_2)

print(f"Computed {len(sift_descriptors_2)} SIFT descriptors at ORB keypoint locations")

out_img_orb_desc = input_image.copy()
cv2.drawKeypoints(out_img_orb_desc, orb_keypoints_2, out_img_orb_desc, flags=sift_viz_flag)
show_image(out_img_orb_desc, "ORB Keypoints")

Now let's check the difference between ORB descriptors and SIFT descriptors computed at the same ORB keypoint locations.

In [ ]:
# TODO: choose an index of a descriptor to visualize and compare
DESCRIPTOR_IDX = 100

if DESCRIPTOR_IDX >= min(len(orb_descriptors_2), len(sift_descriptors_2)):
	raise ValueError(f"DESCRIPTOR_IDX {DESCRIPTOR_IDX} is out of bounds"
				  f" for number of descriptors {min(len(orb_descriptors_2), len(sift_descriptors_2))}")

out_img_1_desc = input_image.copy()
cv2.drawKeypoints(out_img_1_desc, [orb_keypoints_2[DESCRIPTOR_IDX]], out_img_1_desc, flags=sift_viz_flag)

# now crop image around keypoint
x, y = orb_keypoints_2[DESCRIPTOR_IDX].pt
s = orb_keypoints_2[DESCRIPTOR_IDX].size
s *= 2
min_x = max(0, int(x - s/2))
min_y = max(0, int(y - s/2))
max_x = min(input_image.shape[1], int(x + s/2))
max_y = min(input_image.shape[0], int(y + s/2))
crop_img = out_img_1_desc[min_y:max_y, min_x:max_x]
show_images([out_img_1_desc, crop_img], [f"ORB Keypoint n. {DESCRIPTOR_IDX}", f"Cropped Image at keypoint n. {DESCRIPTOR_IDX}"])

orb_descriptor = orb_descriptors_2[DESCRIPTOR_IDX]
sift_descriptor = sift_descriptors_2[DESCRIPTOR_IDX]

print(f"ORB descriptor at index {DESCRIPTOR_IDX}: {orb_descriptor}, size: {len(orb_descriptor)}")
print(f"\nSIFT descriptor at index {DESCRIPTOR_IDX}: {sift_descriptor}, size: {len(sift_descriptor)}")

## Conclusion

In this notebook, we have explored the fundamental concepts of feature detection and description. We have seen how to:
- Detect corners using the Harris corner detector.
- Detect and describe features using SIFT and ORB.
- Understand the structure of keypoints and descriptors.
- Compare different descriptors to measure feature similarity.
- Combine different detectors and descriptors.

These are the building blocks for many computer vision applications. In the next notebook, we will see how to use these features to match images.